# HW3 - EE599 Systems for Machine Learning, Fall 2023
University of Southern California

Instructors: Arash Saifhashemi, Murali Annavaram

In this homework assignment, we will ask you to use various methods to implement convolution operation, and then measure and analyze the performance of each method.

## Prepare your Google Drive
- Download `ML_Systems_HW3` zip file from GitHub and unzip the it (you may need to rename the unzipped folder).
- Upload unzipped folder to ``My Drive`` in Google Drive.

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ml-systems-hw3-Junruiethel-master/src')

Mounted at /content/drive


## Verify that you are in the correct working directory.

In [2]:
!pwd

/content/drive/MyDrive/ml-systems-hw3-Junruiethel-master/src


## Create a folder named `build` under `ML_Systems_HW3/src`, which will be used to store executable files.

In [3]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [75]:
!bash compile.csh

Compiled q1_conv2d_naive.cc successfully!
Compiled q2_conv2d_toeplitz.cc successfully!
q3_conv2d_toeplitz_transB.cc: In function ‘int main()’:
q3_conv2d_toeplitz_transB.cc:103:21: error: ‘output’ was not declared in this scope
  103 |   printSumOfOutputs(output, output_c * output_p * output_q);
      |                     ^~~~~~
q3_conv2d_toeplitz_transB.cc:103:29: error: ‘output_c’ was not declared in this scope
  103 |   printSumOfOutputs(output, output_c * output_p * output_q);
      |                             ^~~~~~~~
q3_conv2d_toeplitz_transB.cc:103:40: error: ‘output_p’ was not declared in this scope
  103 |   printSumOfOutputs(output, output_c * output_p * output_q);
      |                                        ^~~~~~~~
q3_conv2d_toeplitz_transB.cc:103:51: error: ‘output_q’ was not declared in this scope
  103 |   printSumOfOutputs(output, output_c * output_p * output_q);
      |                                                   ^~~~~~~~
q3_conv2d_toeplitz_transB.cc:109:31:

In [76]:
!bash run.csh


Running q1_conv2d_naive ...
Virtual Memory (KB): 16716
temp: 59863496
Time taken by function: 30232 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q2_conv2d_toeplitz ...
Virtual Memory (KB): 87904
temp: 59863496
Time taken by function: 12606 milliseconds
Sum of all outputs: 47049932800
-------------------------------------------
Running q3_conv2d_toeplitz_transB ...
run.csh: line 20: ./build/q3_conv2d_toeplitz_transB: No such file or directory
-------------------------------------------
Running q4_conv2d_toeplitz_avx ...
run.csh: line 20: ./build/q4_conv2d_toeplitz_avx: No such file or directory
-------------------------------------------
Running q5_conv2d_toeplitz_avx_openmp ...
run.csh: line 20: ./build/q5_conv2d_toeplitz_avx_openmp: No such file or directory
-------------------------------------------
Running q6_conv2d_toeplitz_blas ...
run.csh: line 20: ./build/q6_conv2d_toeplitz_blas: No such file or directory
---------------------

## Compile and run your code:
* To compile your C++ code, run command `!bash compile.csh`
* To run your executable code, run command `!bash run.csh`
* Check those csh files and modify accordingly when testing your code.
* You can write and test your code on your local machine but make sure you can compile and run them on Colab. Also you should report the performance measured from Colab enverionment.

## Write code and answer all questions in this notebook.
Note that to measure the performance of each code, we want you to flush your cache. All the template files provide code that flush caches, but we need you to find the cache size of your system. Use the command below to display information about your CPU.

In [4]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo
                         logy nonstop_tsc cpuid tsc_known_freq pni pclmulqdq sss
                         e3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes 
                         xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowpref

## Q1
Implement 2D Convolution in the ``q1_conv2d_naive.cc`` using nested for loops. Assume batch size = 1, no padding and stride = 1. Check `util.h` file and understand what each function does. Use the micro `INDEX_4D_TO_1D` to help convert 4d index to 1d index. Measure and report runtime  (in milliseconds) and memory usage (in KB). Manually calculate memory usage and report. Does your calculation match with the measurement?

In [5]:
#Answer to question1
%%writefile q1_conv2d_naive.cc
#include <iostream>
#include <chrono>
#include "../utils.h"
using namespace std;
using namespace std::chrono;

int main(void)
{
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = (32 + 32 + 256 + 55 * 1024) * 1024;  // Sum of all cache levels
;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float *cacheFlush = allocateAlignedFloatArray(cacheSize);
  initializeBuffer(cacheFlush, cacheFloatSize);
  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // Allocate input buffers
  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float *weight = allocateAlignedFloatArray(weightSize);
  float *input = allocateAlignedFloatArray(inputSize);

  // Initialize input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);
  //initializeBuffer(cacheFlush, cacheFloatSize);

  // TODO: calculate output dimension and allocate output buffer
  const unsigned int output_c = weight_o;
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int outputSize = output_c * output_p * output_q;
  float *output = allocateAlignedFloatArray(outputSize);
  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: use nested for loop to compute convolution
  for (unsigned int o = 0; o < weight_o; o++)
  {
    for (unsigned int p = 0; p < output_p; p++)
    {
      for (unsigned int q = 0; q < output_q; q++)
      {
        float value = 0.0;
        for (unsigned int i = 0; i < weight_i; i++)
        {
          for (unsigned int r = 0; r < weight_r; r++)
          {
            for (unsigned int s = 0; s < weight_s; s++)
            {
              value += input[INDEX_4D_TO_1D(0, i, p + r, q + s, 1, input_c, input_h, input_w)] * weight[INDEX_4D_TO_1D(o, i, r, s, weight_o, weight_i, weight_r, weight_s)];
            }
          }
        }
        output[INDEX_4D_TO_1D(0, o, p, q, 1, output_c, output_p, output_q)] = value;
      }
    }
  }
  // for the runtime of this code, please measure convolution only

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // sum all outputs
  float sum = 0.0;
  for (int i = 0; i < 1 * output_c * output_p * output_q; i++)
  {
    sum += output[i];
  }
  cout.precision(20);
  cout << "Sum of all outputs: " << sum << endl;

  // free all allocated buffers
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);

  return 0;
}


Overwriting q1_conv2d_naive.cc


In [8]:
!g++ q1_conv2d_naive.cc -o output_program

In [9]:
!./output_program

Virtual Memory (KB): 16716
temp: 59863496
Time taken by function: 25683 milliseconds
Sum of all outputs: 47243571200


## Q2
Use Img2col algorithm to convert the input matrix and kernel matrix to toeplitz form in file ``q2_conv2d_toeplitz.cc``. Interpret input toeplitz matrix and kernel toeplitz matrix as 2d matrix, and store both of them in row major. Use nested for loops to perform matrix multiplication in `matMul` function and call it in `main` function . There is another micro `INDEX_2D_TO_1D` that helps convert 2d index to 1d index. Measure and report runtime and memory usage. Manually calculate memory usage and report. Does your calculation match with the measurement?

In [50]:
!echo "" > q2_conv2d_toeplitz.cc

In [69]:
#Answer to Question2
%%writefile q2_conv2d_toeplitz.cc
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

void im2col(const float* data_im, int channels, int height, int width,
            int ksize, int stride, float* data_col) {
    int output_h = (height - ksize) / stride + 1;
    int output_w = (width - ksize) / stride + 1;
    int col_length = channels * ksize * ksize;
    for (int c = 0; c < col_length; ++c) {
        int w_offset = c % ksize;
        int h_offset = (c / ksize) % ksize;
        int c_im = c / ksize / ksize;
        for (int h = 0; h < output_h; ++h) {
            for (int w = 0; w < output_w; ++w) {
                int im_row = h_offset + h * stride;
                int im_col = w_offset + w * stride;
                int col_index = (c * output_h + h) * output_w + w;
                data_col[col_index] = data_im[im_col + width * (im_row + height * c_im)];
            }
        }
    }
}

void matMul(const float *A, const float *B, float *C, int A_rows, int A_cols,
            int B_rows, int B_cols)
{
  if (A_cols != B_rows)
  {
    // The matrices can't be multiplied if A's number of columns
    // isn't equal to B's number of rows.
    throw std::invalid_argument(
        "Matrix dimensions mismatch for multiplication");
  }

  // TODO: implement matmul using 3 loops
  for (int i = 0; i < A_rows; ++i)
  {
    for (int j = 0; j < B_cols; ++j)
    {
      float sum = 0;
      for (int k = 0; k < A_cols; ++k)
      {
        sum += A[INDEX_2D_TO_1D(i, k, A_rows, A_cols)] * B[INDEX_2D_TO_1D(k, j, B_rows, B_cols)];
      }
      C[INDEX_2D_TO_1D(i, j, A_rows, B_cols)] = sum;
    }
  }
}

int main(void)
{
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = (32 + 32 + 256 + 55 * 1024) * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float *cacheFlush = allocateAlignedFloatArray(cacheSize);
  initializeBuffer(cacheFlush, cacheFloatSize);


  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;
  // dimension of the output
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int output_c = weight_o;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float *weight = allocateAlignedFloatArray(weightSize);
  float *input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_h = input_h - weight_r + 1;
  const unsigned int output_w = input_w - weight_s + 1;
  const unsigned int toeplitzOutputSize = weight_o * output_h * output_w;
  float *toeplitzOutput = allocateAlignedFloatArray(toeplitzOutputSize);
  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int toeplitzInputRows = output_h * output_w;
  const unsigned int toeplitzInputCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzInputSize = toeplitzInputRows * toeplitzInputCols;
  float *toeplitzInput = allocateAlignedFloatArray(toeplitzInputSize);
  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int toeplitzWeightRows = weight_o;
  const unsigned int toeplitzWeightCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzWeightSize = toeplitzWeightRows * toeplitzWeightCols;
  float *toeplitzWeight = allocateAlignedFloatArray(toeplitzWeightSize);
  const unsigned int temp_output_size = toeplitzInputRows * toeplitzWeightRows;
  float *temp_output = allocateAlignedFloatArray(temp_output_size);
  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;
  float *output = allocateAlignedFloatArray(toeplitzOutputSize);
  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in row
  // major order
  im2col(input, input_c, input_h, input_w, weight_r, 1, toeplitzInput);
  // Flatten the filter for Toeplitz matrix
  for (int oc = 0; oc < weight_o; ++oc) {
        for (int ic = 0; ic < input_c; ++ic) {
            for (int i = 0; i < weight_r; ++i) {
                for (int j = 0; j < weight_s; ++j) {
                    int out_index = j + weight_s * (i + weight_r * ic);
                    toeplitzWeight[INDEX_2D_TO_1D(oc, out_index, weight_o, input_c * weight_r * weight_s)] = weight[oc * input_c * weight_r * weight_s + ic * weight_r * weight_s + i * weight_s + j];
                }
            }
        }
    }
  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toeplitz matrices, you can create a temp
  // buffer to store output_toeplitz measure runtime of this code only
  // Allocate memory for temp_output using the provided function
  matMul(toeplitzInput, toeplitzWeight, temp_output, toeplitzInputRows, toeplitzInputCols, toeplitzWeightCols, toeplitzWeightRows);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toeplitz y into row major
  for (int oc = 0; oc < output_c; ++oc) {
    for (int op = 0; op < output_p; ++op) {
        for (int oq = 0; oq < output_q; ++oq) {
            int flat_index = oc * output_p * output_q + op * output_q + oq;
            output[INDEX_2D_TO_1D(oc, op * output_q + oq, output_c, output_p * output_q)] = temp_output[flat_index];
        }
    }
  }

  printSumOfOutputs(output, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);
  deallocateAlignedFloatArray(temp_output);
  return 0;
}

Overwriting q2_conv2d_toeplitz.cc


In [70]:
!g++ q2_conv2d_toeplitz.cc -o output_program_02

In [71]:
!./output_program_02

Virtual Memory (KB): 87904
temp: 59863496
Time taken by function: 11396 milliseconds
Sum of all outputs: 47049932800


## Q3
For `q3_conv2d_toeplitz_tranB.cc`, repeat procedures in Q2, but store kernel toeplitz matrix in column major and modify `matMul` function accordingly. Measure and report the runtime and memory usage.

In [78]:
!echo "" > q3_conv2d_toeplitz_transB.cc

In [87]:
#Answer to Question3
%%writefile q3_conv2d_toeplitz_transB.cc
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

void im2col(const float* data_im, int channels, int height, int width,
            int ksize, int stride, float* data_col) {
    int output_h = (height - ksize) / stride + 1;
    int output_w = (width - ksize) / stride + 1;
    int col_length = channels * ksize * ksize;
    for (int c = 0; c < col_length; ++c) {
        int w_offset = c % ksize;
        int h_offset = (c / ksize) % ksize;
        int c_im = c / ksize / ksize;
        for (int h = 0; h < output_h; ++h) {
            for (int w = 0; w < output_w; ++w) {
                int im_row = h_offset + h * stride;
                int im_col = w_offset + w * stride;
                int col_index = (c * output_h + h) * output_w + w;
                data_col[col_index] = data_im[im_col + width * (im_row + height * c_im)];
            }
        }
    }
}

void matMulTransB(const float* A, const float* B, float* C, int A_rows, int A_cols, int B_rows, int B_cols) {
  if (A_cols != B_cols) {
    throw std::invalid_argument("Matrix dimensions mismatch for multiplication");
  }

  for (int i = 0; i < A_rows; ++i) {
    for (int j = 0; j < B_rows; ++j) {
      float sum = 0;
      for (int k = 0; k < A_cols; ++k) {
        sum += A[INDEX_2D_TO_1D(i, k, A_rows, A_cols)] * B[INDEX_2D_TO_1D(j, k, B_rows, B_cols)];
      }
      C[INDEX_2D_TO_1D(i, j, A_rows, B_rows)] = sum;
    }
  }
}

int main(void)
{
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = (32 + 32 + 256 + 55 * 1024) * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float *cacheFlush = allocateAlignedFloatArray(cacheSize);
  initializeBuffer(cacheFlush, cacheFloatSize);


  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;
  // dimension of the output
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int output_c = weight_o;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float *weight = allocateAlignedFloatArray(weightSize);
  float *input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_h = input_h - weight_r + 1;
  const unsigned int output_w = input_w - weight_s + 1;
  const unsigned int toeplitzOutputSize = weight_o * output_h * output_w;
  float *toeplitzOutput = allocateAlignedFloatArray(toeplitzOutputSize);
  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int toeplitzInputRows = output_h * output_w;
  const unsigned int toeplitzInputCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzInputSize = toeplitzInputRows * toeplitzInputCols;
  float *toeplitzInput = allocateAlignedFloatArray(toeplitzInputSize);
  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int toeplitzWeightRows = weight_o;
  const unsigned int toeplitzWeightCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzWeightSize = toeplitzWeightRows * toeplitzWeightCols;
  float *toeplitzWeight = allocateAlignedFloatArray(toeplitzWeightSize);
  const unsigned int temp_output_size = toeplitzInputRows * toeplitzWeightRows;
  float *temp_output = allocateAlignedFloatArray(temp_output_size);
  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;
  float *output = allocateAlignedFloatArray(toeplitzOutputSize);
  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in row
  // major order
  im2col(input, input_c, input_h, input_w, weight_r, 1, toeplitzInput);
  // Flatten the filter for Toeplitz matrix in column major format
  for (int oc = 0; oc < weight_o; ++oc) {
     for (int ic = 0; ic < input_c; ++ic) {
        for (int i = 0; i < weight_r; ++i) {
            for (int j = 0; j < weight_s; ++j) {
                int out_index = i + weight_r * (j + weight_s * ic);
                toeplitzWeight[INDEX_2D_TO_1D(out_index, oc, input_c * weight_r * weight_s, weight_o)] = weight[oc * input_c * weight_r * weight_s + ic * weight_r * weight_s + i * weight_s + j];
              }
          }
      }
  }

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toeplitz matrices, you can create a temp
  // buffer to store output_toeplitz measure runtime of this code only
  // Allocate memory for temp_output using the provided function
 matMulTransB(toeplitzInput, toeplitzWeight, temp_output, toeplitzInputRows, toeplitzInputCols, toeplitzWeightRows, toeplitzWeightCols);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toeplitz y into row major
  for (int oc = 0; oc < output_c; ++oc) {
    for (int op = 0; op < output_p; ++op) {
        for (int oq = 0; oq < output_q; ++oq) {
            int flat_index = oc * output_p * output_q + op * output_q + oq;
            output[INDEX_2D_TO_1D(oc, op * output_q + oq, output_c, output_p * output_q)] = temp_output[flat_index];
        }
    }
  }

  printSumOfOutputs(output, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);
  deallocateAlignedFloatArray(temp_output);
  return 0;
}

Overwriting q3_conv2d_toeplitz_transB.cc


In [88]:
!g++ q3_conv2d_toeplitz_transB.cc -o output_program_03

In [89]:
!./output_program_03

Virtual Memory (KB): 87904
temp: 59863496
Time taken by function: 8456 milliseconds
Sum of all outputs: 47961726976


## Q4
For ``q4_conv2d_toeplitz_avx.cc``, use Intel AVX (Advanced Vector Extensions) instruction set to perform matmul operation. Intel AVX instructions are Single Instruction Multiple Data (SIMD) instructions that can process 8 floating-point operands in a single instruction. Store input toeplitz matrix in row major and kernel toeplitz matrix in column major. An example of using Intel AVX is given in file ``examples/example_vectorsum_simd.cc``. Measure and report runtime and memory usage.

Hint 1: Use `_mm256_add_ps` and `_mm256_mul_ps` instructions.

Hint 2: If a vector is not divisible by 8, the remaining elements in the vector should not be processed by SIMD instruction. Instead, use normal scalar operations.

In [79]:
#Answer to Question4
%%writefile q4_conv2d_toeplitz_avx.cc
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

void im2col(const float* data_im, int channels, int height, int width,
            int ksize, int stride, float* data_col) {
    int output_h = (height - ksize) / stride + 1;
    int output_w = (width - ksize) / stride + 1;
    int col_length = channels * ksize * ksize;
    for (int c = 0; c < col_length; ++c) {
        int w_offset = c % ksize;
        int h_offset = (c / ksize) % ksize;
        int c_im = c / ksize / ksize;
        for (int h = 0; h < output_h; ++h) {
            for (int w = 0; w < output_w; ++w) {
                int im_row = h_offset + h * stride;
                int im_col = w_offset + w * stride;
                int col_index = (c * output_h + h) * output_w + w;
                data_col[col_index] = data_im[im_col + width * (im_row + height * c_im)];
            }
        }
    }
}

void matMulAVXTransB(const float* A, const float* B, float* C, int A_rows,
                     int A_cols, int B_rows, int B_cols) {
    if (A_cols != B_rows) {
        throw std::invalid_argument("Matrix dimensions mismatch for multiplication");
    }

    int A_cols_divided_by_8 = A_cols - (A_cols % 8);
    for (int i = 0; i < A_rows; ++i) {
        for (int j = 0; j < B_cols; ++j) {
            C[i * B_cols + j] = 0.0;

            __m256 sum = _mm256_setzero_ps();
            for (int k = 0; k < A_cols_divided_by_8; k += 8) {
                __m256 a = _mm256_load_ps(&A[i*A_cols + k]);
                __m256 b = _mm256_load_ps(&B[j*B_rows + k]);
                sum = _mm256_add_ps(sum, _mm256_mul_ps(a, b));
            }
            float* buffer = (float*) _mm_malloc(8 * sizeof(float), 32);
            _mm256_store_ps(buffer, sum);
            for (int index = 0; index < 8; ++index) {
            C[i * B_cols + j] += buffer[index];
            }
            for (int k = A_cols_divided_by_8; k < A_cols; ++k) {
                C[i * B_cols + j] += A[i * A_cols + k] * B[j * B_rows + k];
            }
            _mm_free(buffer);
        }
    }

}


int main(void)
{
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = (32 + 32 + 256 + 55 * 1024) * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float *cacheFlush = (float*) aligned_alloc(256, (32 + 32 + 256 + 55 * 1024) * 1024);
  initializeBuffer(cacheFlush, cacheFloatSize);


  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;
  // dimension of the output
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int output_c = weight_o;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float *weight = (float*) aligned_alloc(256, weightSize * sizeof(float));
  float *input = (float*) aligned_alloc(256, inputSize * sizeof(float));

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_h = input_h - weight_r + 1;
  const unsigned int output_w = input_w - weight_s + 1;
  const unsigned int toeplitzOutputSize = weight_o * output_h * output_w;
  float* toeplitzOutput = (float*) aligned_alloc(256, toeplitzOutputSize * sizeof(float));
  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int toeplitzInputRows = output_h * output_w;
  const unsigned int toeplitzInputCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzInputSize = toeplitzInputRows * toeplitzInputCols;
  float* toeplitzInput = (float*) aligned_alloc(256, toeplitzInputSize * sizeof(float));
  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int toeplitzWeightRows = weight_o;
  const unsigned int toeplitzWeightCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzWeightSize = toeplitzWeightRows * toeplitzWeightCols;
  float* toeplitzWeight = (float*) aligned_alloc(256, toeplitzWeightSize * sizeof(float));
  const unsigned int temp_output_size = toeplitzInputRows * toeplitzWeightRows;
  float* temp_output = (float*) aligned_alloc(256, temp_output_size * sizeof(float));
  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;
  float* output = (float*) aligned_alloc(256, toeplitzOutputSize * sizeof(float));
  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in row
  // major order
  im2col(input, input_c, input_h, input_w, weight_r, 1, toeplitzInput);
  // Flatten the filter for Toeplitz matrix in column major format
  for (int oc = 0; oc < weight_o; ++oc) {
     for (int ic = 0; ic < input_c; ++ic) {
        for (int i = 0; i < weight_r; ++i) {
            for (int j = 0; j < weight_s; ++j) {
                int out_index = i + weight_r * (j + weight_s * ic);
                toeplitzWeight[INDEX_2D_TO_1D(out_index, oc, input_c * weight_r * weight_s, weight_o)] = weight[oc * input_c * weight_r * weight_s + ic * weight_r * weight_s + i * weight_s + j];
              }
          }
      }
  }

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toeplitz matrices, you can create a temp
  // buffer to store output_toeplitz measure runtime of this code only
  // Allocate memory for temp_output using the provided function
  matMulAVXTransB(toeplitzInput, toeplitzWeight, temp_output, toeplitzInputRows, toeplitzInputCols, toeplitzWeightCols, toeplitzWeightRows);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat kernel toeplitz y into column major
  for (int oc = 0; oc < weight_o; ++oc) {
    for (int ic = 0; ic < input_c; ++ic) {
        for (int i = 0; i < weight_r; ++i) {
            for (int j = 0; j < weight_s; ++j) {
                int out_index = j + weight_s * (i + weight_r * ic);
                toeplitzWeight[INDEX_2D_TO_1D(oc, out_index, weight_o, input_c * weight_r * weight_s)] = weight[oc * input_c * weight_r * weight_s + ic * weight_r * weight_s + i * weight_s + j];
            }
        }
    }
}


  printSumOfOutputs(output, output_c * output_p * output_q);

  // free all allocated buffers
  free(input);
  free(toeplitzOutput);
  free(toeplitzInput);
  free(toeplitzWeight);
  free(temp_output);
  free(output);

  return 0;
}

Overwriting q4_conv2d_toeplitz_avx.cc


In [80]:
!g++ q4_conv2d_toeplitz_avx.cc -mavx -o output_program_04

In [81]:
!./output_program_04

Virtual Memory (KB): 87904
temp: 59863496
Time taken by function: 2620 milliseconds
Sum of all outputs: 0


## Q5

### Part 1
In file ``q5_conv2d_toeplitz_avx_openmp.cc``, futher optimize the AVX matmul operation using ``OpenMP`` library, which enables multi-threaded parallel computing automatically through a simple and flexible interface. An example of using ``OpenMP`` is given in file ``examples/example_vectorsum_simd_omd.cc``. Run the code with different number of threads. Measure and report the runtime for number of threads = [2, 4, 8].


### Part 2 (Optional - No Credit)

In file ``q5_optional_conv2d_toeplitz_avx_multi_thread.cc``, instead of using `OpenMP`, use the C++ standard `<thread>` library to implement multi-threaded AVX matmul operation. You can refer to [this video](https://youtu.be/3aqxaZsvn80?si=1QEE580e2vLmrqPO) to learn about multi threading in C++.


## Q6

In file ``q6_conv2d_toeplitz_blas.cc``, use `BLAS` library to implement matmul operation. You can decide the storage format for input and kernel toeplitz matrices, and make sure you set input arguments of `cblas_sgemm` accordingly. An example is given in file ``examples/example_matmul_blas.cc``. Search online document of `cblas_sgemm` API if you are unclear about the mearning of each of its input argument. Follow the example and and finish your own code. Measure and report runtime and memory usage.

In [113]:
#Answer to Question6
%%writefile q6_conv2d_toeplitz_blas.cc
extern "C" {
#include "../cblas.h"
}
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

void im2col(const float* data_im, int channels, int height, int width,
    int ksize, int stride, float* data_col) {
    int output_h = (height - ksize) / stride + 1;
    int output_w = (width - ksize) / stride + 1;
    int col_length = channels * ksize * ksize;
    for (int c = 0; c < col_length; ++c) {
        int w_offset = c % ksize;
        int h_offset = (c / ksize) % ksize;
        int c_im = c / ksize / ksize;
        for (int h = 0; h < output_h; ++h) {
            for (int w = 0; w < output_w; ++w) {
                int im_row = h_offset + h * stride;
                int im_col = w_offset + w * stride;
                int col_index = (c * output_h + h) * output_w + w;
                data_col[col_index] = data_im[im_col + width * (im_row + height * c_im)];
            }
        }
    }
}

void matMulBLAS(const float* A, const float* B, float* C, int A_rows, int A_cols, int B_rows, int B_cols) {
if (A_cols != B_rows) {
    std::cout << "Matrix A Columns: " << A_cols << std::endl;
    std::cout << "Matrix B Rows: " << B_rows << std::endl;
    throw std::invalid_argument("Matrix dimensions mismatch for multiplication");
}
    std::cout << "Matrix A Dimensions: " << A_rows << "x" << A_cols << std::endl;
    std::cout << "Matrix B Dimensions: " << B_rows << "x" << B_cols << std::endl;
    int M = A_rows;
    int N = B_cols;
    int K = A_cols;

    float alpha = 1.0f;
    float beta = 0.0f;
    cblas_sgemm(CblasRowMajor, CblasNoTrans, CblasTrans,
                M, N, K, alpha,
                A, K,
                B, N, beta,
                C, N);
}

int main(void)
{
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = (32 + 32 + 256 + 55 * 1024) * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float *cacheFlush = allocateAlignedFloatArray(cacheSize);
  initializeBuffer(cacheFlush, cacheFloatSize);


  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;
  // dimension of the output
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int output_c = weight_o;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float *weight = allocateAlignedFloatArray(weightSize);
  float *input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_h = input_h - weight_r + 1;
  const unsigned int output_w = input_w - weight_s + 1;
  const unsigned int toeplitzOutputSize = weight_o * output_h * output_w;
  float *toeplitzOutput = allocateAlignedFloatArray(toeplitzOutputSize);
  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int toeplitzInputRows = output_h * output_w;
  const unsigned int toeplitzInputCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzInputSize = toeplitzInputRows * toeplitzInputCols;
  float *toeplitzInput = allocateAlignedFloatArray(toeplitzInputSize);
  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int toeplitzWeightRows = weight_o;
  const unsigned int toeplitzWeightCols = input_c * weight_r * weight_s;
  const unsigned int toeplitzWeightSize = toeplitzWeightRows * toeplitzWeightCols;
  float *toeplitzWeight = allocateAlignedFloatArray(toeplitzWeightSize);
  const unsigned int temp_output_size = toeplitzInputRows * toeplitzWeightRows;
  float *temp_output = allocateAlignedFloatArray(temp_output_size);
  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;
  float *output = allocateAlignedFloatArray(toeplitzOutputSize);
  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in row
  // major order
  im2col(input, input_c, input_h, input_w, weight_r, 1, toeplitzInput);
  // Flatten the filter for Toeplitz matrix in column major format
  for (int oc = 0; oc < weight_o; ++oc) {
    for (int ic = 0; ic < input_c; ++ic) {
        for (int i = 0; i < weight_r; ++i) {
            for (int j = 0; j < weight_s; ++j) {
                int out_index = i + weight_r * (j + weight_s * ic);
                toeplitzWeight[INDEX_2D_TO_1D(oc, out_index, weight_o, input_c * weight_r * weight_s)] =
                    weight[oc * input_c * weight_r * weight_s + ic * weight_r * weight_s + i * weight_s + j];
            }
         }
      }
   }


  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toeplitz matrices, you can create a temp
  // buffer to store output_toeplitz measure runtime of this code only
  // Allocate memory for temp_output using the provided function
  std::cout << "Matrix A Rows: " << toeplitzInputRows << std::endl;
  std::cout << "Matrix B Columns: " << toeplitzWeightCols << std::endl;
  matMulBLAS(toeplitzInput, toeplitzWeight, temp_output, toeplitzInputRows, toeplitzInputCols, toeplitzWeightRows, toeplitzWeightCols);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toeplitz y into row major
  for (int oc = 0; oc < output_c; ++oc) {
    for (int op = 0; op < output_p; ++op) {
        for (int oq = 0; oq < output_q; ++oq) {
            int flat_index = oc * output_p * output_q + op * output_q + oq;
            output[INDEX_2D_TO_1D(oc, op * output_q + oq, output_c, output_p * output_q)] = temp_output[flat_index];
        }
    }
  }

  printSumOfOutputs(output, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);
  deallocateAlignedFloatArray(temp_output);
  return 0;
}

Overwriting q6_conv2d_toeplitz_blas.cc


In [114]:
!g++ q6_conv2d_toeplitz_blas.cc -lcblas -o output_program_06

In [115]:
!./output_program_06

Virtual Memory (KB): 87904
temp: 59863496
Matrix A Rows: 15876
Matrix B Columns: 1152
Matrix A Columns: 1152
Matrix B Rows: 128
terminate called after throwing an instance of 'std::invalid_argument'
  what():  Matrix dimensions mismatch for multiplication


## Q7  

### Part 1


Analyze the performance differences between Q1-Q6. Explain what constitutes the performance difference between each implementations and why `BLAS` library is super fast.

### Part 2: Using Google Benchmark (Optional - No Credit)
As an optional practice, you can measure the runtime of each matrix calculation method more accurately using Google benchmark.

A better way to measure performance of a funcion in C++ is to use Google Benchmark. You can refer to [this video](https://youtu.be/9VKR8u9odrA?si=xSInuzT5uMBOKAbP) to familiarize yourself with this package.




## Upload files to GitHub
Make sure upload your final C++ code and this IPython notebook to GitHub Repo either mannully or through git commands.